In [39]:
import requests

url = "https://api.linnworks.net/api/Auth/AuthorizeByApplication"

payload = {
    "ApplicationId": "2fd8240c-2d8c-4de5-b08d-7d1c96600b03",
    "ApplicationSecret": "73b1e4f1-b430-4fa5-8772-dcbc8420fdee",
    "Token": "22288bc96193dc64fa19fed5953965cb"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

# Parse JSON response
data = response.json()

# Assuming the token is inside a key named 'Token' or 'AccessToken' or similar:
YOUR_ACCESS_TOKEN = data.get("Token")

print(YOUR_ACCESS_TOKEN)


a348732f-fd8b-4601-9659-7749ecf9c4fb


In [16]:
YOUR_ACCESS_TOKEN

'38837d53-801d-48b5-b025-841b2b90bbf4'

In [12]:
import requests
import pandas as pd
from datetime import datetime

#url="https://eu-ext.linnworks.net/api/PurchaseOrder/Get_PurchaseOrder"
url = "https://eu-ext.linnworks.net/api/PurchaseOrder/Search_PurchaseOrders2"

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": access_token  # Your valid access token
}

payload = {
    "searchParameters": {
        "DateFrom": "2000-01-01T00:00:00",  # Very early to get all
        "DateTo": datetime.now().isoformat()
    },
    "entriesPerPage": 100,
    "pageNumber": 1
}

all_orders = []

while True:
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break

    data = response.json()
    orders = data.get("Result", [])
    all_orders.extend(orders)

    total_pages = data.get("TotalPages", 1)
    current_page = data.get("CurrentPageNumber", 1)

    print(f"Fetched page {current_page}/{total_pages}, {len(orders)} orders")

    if current_page >= total_pages:
        break

    payload["pageNumber"] += 1

# Convert to DataFrame with all fields
df = pd.DataFrame(all_orders)



Fetched page 1/4, 100 orders
Fetched page 2/4, 100 orders
Fetched page 3/4, 100 orders
Fetched page 4/4, 70 orders


In [7]:
df.head()

,pkPurchaseID,fkSupplierId,fkLocationId,ExternalInvoiceNumber,Status,Currency,SupplierReferenceNumber,Locked,LineCount,DeliveredLinesCount,...,PostagePaid,TotalCost,taxPaid,ShippingTaxRate,ConversionRate,ConvertedShippingCost,ConvertedShippingTax,ConvertedOtherCost,ConvertedOtherTax,ConvertedGrandTotal
0,e34457e6-2220-49d4-8768-d39e03f80478,00000000-0000-0000-0000-000000000000,e05a5ecc-f51e-41aa-8872-12a1f5b6aa33,PO0511243992,DELIVERED,GBP,,False,3,3,...,0.0,55576.224,9262.704,0.0,1.0,0.0,0.0,0.0,0.0,55576.22
1,517f92ff-2270-4d85-9331-a18cb2c1fbed,9b6ba0e1-4827-4d35-961f-dfb602c609b8,4c138fbe-81b9-4f8a-97b9-1469f35825db,PO05112428017,DELIVERED,GBP,MIL001,False,4,4,...,0.0,642.456,107.076,0.0,1.0,0.0,0.0,0.0,0.0,642.46
2,875ba104-2605-4ced-aa3f-51d2d289eadc,00000000-0000-0000-0000-000000000000,4c138fbe-81b9-4f8a-97b9-1469f35825db,PO08112429560,DELIVERED,GBP,,False,1,1,...,0.0,1.200,0.200,0.0,1.0,0.0,0.0,0.0,0.0,1.20
3,a500b35e-cab5-4719-8288-3f188a58dd17,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,PO11112456020,DELIVERED,GBP,,False,1,0,...,0.0,467.160,77.860,0.0,1.0,0.0,0.0,0.0,0.0,467.16
4,9936293b-716e-474c-8f82-845f9bd8f53b,50de353c-4ad3-43fd-9527-a8d096876932,cf4c4f85-fc1e-494e-855d-759f81258d3d,PO13112433956,DELIVERED,GBP,,False,1,1,...,0.0,24624.000,4104.000,0.0,1.0,0.0,0.0,0.0,0.0,24624.00


In [30]:
import requests
import math

access_token = "38837d53-801d-48b5-b025-841b2b90bbf4"
url = "https://eu-ext.linnworks.net/api/Stock/GetStockItemsFull"

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": access_token
}

entries_per_page = 100
page_number = 1
total_items = 0

while True:
    payload = {
        "keyword": "",
        "loadCompositeParents": True,
        "loadVariationParents": True,
        "entriesPerPage": entries_per_page,
        "pageNumber": page_number,
        "dataRequirements": ["StockLevels"],
        "searchTypes": ["SKU"]
    }

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code != 200:
        raise RuntimeError(f"API error {response.status_code}: {response.text}")

    data = response.json()
    count = len(data)
    total_items += count

    if not data:  # no more items
        break

    page_number += 1

# Calculate total pages based on entries_per_page
total_pages = math.ceil(total_items / entries_per_page)
print(f"Total items fetched: {total_items}")
print(f"Estimated total pages: {total_pages}")


RuntimeError: API error 429: "API calls quota exceeded! Maximum admitted 150 per Minute."

In [26]:
df.head()

,Suppliers,StockLevels,ItemChannelDescriptions,ItemExtendedProperties,ItemChannelTitles,ItemChannelPrices,Images,ItemNumber,ItemTitle,BarcodeNumber,...,Height,Width,Depth,Weight,CreationDate,InventoryTrackingType,BatchNumberScanRequired,SerialNumberScanRequired,StockItemId,StockItemIntId
0,[],[{'Location': {'StockLocationId': '00000000-00...,[],[],[],[],[],027-0001C,,,...,0.0,0.0,0.0,0.000,2025-02-27T12:47:03.593Z,0,False,False,14f067be-475e-4ddf-907e-8461b1ec3f3b,0
1,[],[{'Location': {'StockLocationId': '00000000-00...,[],[],[],[],[],048-0001A,,99999,...,0.0,0.0,0.0,0.000,2025-02-27T12:47:03.593Z,0,False,False,4f177340-6b8e-45fc-a908-4918f1352b57,0
2,[],[{'Location': {'StockLocationId': '00000000-00...,[],[],[],[],[],100001,Makita DTD152Z 18v LXT Cordless Impact Driver ...,,...,205.0,165.0,90.0,1.107,2025-03-30T20:47:04.91Z,0,False,False,7d9329bc-1def-43d5-8bce-9784ce9009fd,0
3,[],[{'Location': {'StockLocationId': '00000000-00...,[],[],[],[],[],1619BZ0100,,0000000000000,...,0.0,0.0,0.0,0.000,2025-02-27T12:47:03.593Z,0,False,False,88b8ea19-3bd7-40ea-b18b-d46e4c98c007,0
4,[],[{'Location': {'StockLocationId': '00000000-00...,[],[],[],[],[],578623,578623:Incorrect SKU,,...,0.0,0.0,0.0,0.000,2025-06-22T14:38:35.23Z,0,False,False,ae3e3772-2dcb-444a-be78-b91660fc22ff,0


In [29]:
df.count()

Suppliers                   200
StockLevels                 200
ItemChannelDescriptions     200
ItemExtendedProperties      200
ItemChannelTitles           200
ItemChannelPrices           200
Images                      200
ItemNumber                  200
ItemTitle                   200
BarcodeNumber               200
MetaData                    200
IsVariationParent           200
isBatchedStockType          200
PurchasePrice               200
TaxRate                     200
PostalServiceId             200
CategoryId                  200
CategoryName                200
PackageGroupId              200
Height                      200
Width                       200
Depth                       200
Weight                      200
CreationDate                200
InventoryTrackingType       200
BatchNumberScanRequired     200
SerialNumberScanRequired    200
StockItemId                 200
StockItemIntId              200
dtype: int64

In [31]:
df.columns

Index(['Suppliers', 'StockLevels', 'ItemChannelDescriptions',
       'ItemExtendedProperties', 'ItemChannelTitles', 'ItemChannelPrices',
       'Images', 'ItemNumber', 'ItemTitle', 'BarcodeNumber', 'MetaData',
       'IsVariationParent', 'isBatchedStockType', 'PurchasePrice', 'TaxRate',
       'PostalServiceId', 'CategoryId', 'CategoryName', 'PackageGroupId',
       'Height', 'Width', 'Depth', 'Weight', 'CreationDate',
       'InventoryTrackingType', 'BatchNumberScanRequired',
       'SerialNumberScanRequired', 'StockItemId', 'StockItemIntId'],
      dtype='object')

In [33]:
df.tail()

,Suppliers,StockLevels,ItemChannelDescriptions,ItemExtendedProperties,ItemChannelTitles,ItemChannelPrices,Images,ItemNumber,ItemTitle,BarcodeNumber,...,Height,Width,Depth,Weight,CreationDate,InventoryTrackingType,BatchNumberScanRequired,SerialNumberScanRequired,StockItemId,StockItemIntId
195,[],[{'Location': {'StockLocationId': '00000000-00...,[],[],[],[],[],ARMFRLK2-240,Armorgard FRLK2-240 Forma-Stor Lighting Kit 24...,5060452083247,...,0.0,0.0,0.0,0.0,2025-01-28T06:00:49.067Z,0,False,False,e2fe7f48-1ef0-4c41-b713-8317f13eab0e,0
196,[],[{'Location': {'StockLocationId': 'd3fc787a-59...,[],[],[],[],[],ARMFRLK-240,,,...,0.0,0.0,0.0,0.0,2025-07-03T12:43:32.217Z,0,False,False,d0bddd35-3dab-4829-8955-2e67e45c6a9a,0
197,[],[{'Location': {'StockLocationId': 'd3fc787a-59...,[],[],[],[],[],ARMFRLK3,,,...,0.0,0.0,0.0,0.0,2025-07-03T12:43:32.217Z,0,False,False,78fd1a51-bbc3-429d-b0a0-9d4e72364eb6,0
198,[],[{'Location': {'StockLocationId': '00000000-00...,[],[],[],[],[],ARMFRR4,"Armorgard FRR4 Forma-Stor Ramp For Forma-Stor,...",5060452082646,...,195.0,750.0,1185.0,37.0,2025-02-12T06:00:37.69Z,0,False,False,b8b0fad9-3219-4fea-bef5-f47c5390be55,0
199,[],[{'Location': {'StockLocationId': '00000000-00...,[],[],[],[],[],ARMFRS2,Armorgard FRS2 Forma-Stor Shelf For Forma-Stor,5060452082455,...,30.0,520.0,1970.0,20.0,2025-01-28T06:00:49.067Z,0,False,False,653a39bb-9ac0-4581-9b11-8e10beee13d1,0


In [34]:
pd.set_option('display.max_colwidth', None)

In [38]:
df['StockLevels'].tail(1)

199    [{'Location': {'StockLocationId': '00000000-0000-0000-0000-000000000000', 'StockLocationIntId': 1, 'LocationName': 'Default', 'LocationTag': '', 'IsFulfillmentCenter': False, 'IsWarehouseManaged': False, 'BinRack': ''}, 'StockLevel': 0, 'StockValue': 0.0, 'MinimumLevel': 4, 'InOrderBook': 0, 'Due': 0, 'JIT': False, 'InOrders': 0, 'Available': 0, 'UnitCost': 0.0, 'AutoAdjust': False, 'LastUpdateDate': '2025-06-23T20:30:46.077Z', 'LastUpdateOperation': 'Imported from file', 'rowid': '00000000-0000-0000-0000-000000000000', 'PendingUpdate': False, 'StockItemPurchasePrice': 0.0, 'StockItemId': '653a39bb-9ac0-4581-9b11-8e10beee13d1', 'StockItemIntId': 0}, {'Location': {'StockLocationId': 'd3fc787a-596a-43a5-aaba-8baf9f3554fd', 'StockLocationIntId': 39, 'LocationName': 'Armorgard', 'LocationTag': '', 'IsFulfillmentCenter': True, 'IsWarehouseManaged': False, 'BinRack': ''}, 'StockLevel': 5, 'StockValue': 736.35, 'MinimumLevel': 4, 'InOrderBook': 0, 'Due': 0, 'JIT': False, 'InOrders': 0,